# Add logo to a batch of images

Haideți să vedem cum putem automatiza munca plictisitoare pe care o presupune  adăugarea unui logo la un grup de imagini. După cum vă imaginați, a face asta de mână într-un program free poate dura destul de mult, și, în plus, este o treabă destul de plictisitoare.
Iar programe precum Photoshop sau Lightroom costă destul de mult și nu sunt chiar super-ușor de folosit instant.

PASUL 1: Pentru ce imagini adăugăm logo?
    - Cu alte cuvinte, vom cere user-ului să introducă de la tastatură:
        - calea către directorul unde se află imaginile de procesat
        - care sunt extensiile de luat în calcul pentru procesare din acest director
          NOTĂ: 
            de eliminat acele extensii care nu sunt extensii comune de imagini și de avertizat user-ul că 
            acea extensie nu va fi luată în calcul.

In [1]:
# TODO 1.1. - import modules useful in current dir change and containing image processing useful methods
from PIL import Image
import os

# TODO 1.2. - save in a variable the path to the directory where we want to take the images from
# "C:\Users\mgaman\Pictures\pycodette_extra_reduced"
from_dir = input("Introduce the path to the directory where the images to be processed are: ")
os.chdir(from_dir)

# TODO 1.3. - Ask user to specify extensions desired
extensions = []
accepted_ext = [".jpg", ".jpeg", ".png", ".bmp", ".ppm", ".pgm", ".pbm", ".pnm"]
ext = input("Specify the first extension to be considered: ")
while True:
    if ext[0] != '.':
        ext = '.' + ext
    # NOTE: we do not care if the extension is uppercase or lower case. 
    # All of these cases should be considered as valid extensions
    if ext.lower() in accepted_ext:
        extensions.append(ext)
        another = input("Do you have another extension to introduce? yes/[no] ")
        if another == "yes":
            ext = input("Introduce the next extension: ")
            continue
        break
    ext = input("This not a valid image extension. Try again! ")


# TODO 1.4. - Keep a list of image objects all from this dir having one of the the specified extensions  
to_be_proc_list = []
im_names = []
for filename in os.listdir(from_dir):
    for ext in extensions:
        if filename.endswith(ext):
            to_be_proc_list.append(Image.open(filename)) 
            im_names.append(filename)
            continue
# Test        
print(im_names)

Introduce the path to the directory where the images to be processed are: C:\Users\mgaman\Pictures\pycodette_extra_reduced
Specify the first extension to be considered: .JPG
Do you have another extension to introduce? yes/[no] 
['_DSC0643.JPG', '_DSC0644.JPG', '_DSC0648.JPG', '_DSC0649.JPG', '_DSC0655.JPG', '_DSC0656.JPG', '_DSC0659.JPG', '_DSC0661.JPG', '_DSC0663.JPG', '_DSC0664.JPG', '_DSC0667.JPG', '_DSC0697.JPG', '_DSC0699.JPG', '_DSC0700.JPG', '_DSC0701.JPG', '_DSC0703.JPG', '_DSC0706.JPG', '_DSC0708.JPG', '_DSC0711.JPG', '_DSC0712.JPG', '_DSC0715.JPG', '_DSC0733.JPG', '_DSC0736.JPG', '_DSC0777.JPG', '_DSC0779.JPG', '_DSC0796.JPG', '_DSC0797.JPG', '_DSC0801.JPG', '_DSC0803.JPG', '_DSC0805.JPG', '_DSC0806.JPG', '_DSC0807.JPG', '_DSC0808.JPG', '_DSC0809.JPG', '_DSC0812.JPG', '_DSC0813.JPG', '_DSC0814.JPG', '_DSC0821.JPG', '_DSC0824.JPG', '_DSC0846.JPG', '_DSC0850.JPG', '_DSC0954.JPG', '_DSC0955.JPG', '_DSC0956.JPG', '_DSC0957.JPG', '_DSC0958.JPG']


PASUL 2 
    - Unde este logo-ul?
    - vrem să fie mai mare, mai mic, la fel?
    - în care colț al imaginii îl așezăm?
      Răspunsul la această întrebare va presupune, desigur, și a calcula poziția colțului 
      stânga sus a logo-ului relativ la dimensiunile imaginii și la dimensiunile acestuia.
      Însă nu vom face acest lucru la pasul curent, căci este mai eficient să calculăm 
      poziția logo-ului în funcție de proprietățile fiecărei imagini, atunci când vom modifica efectiv
      imaginile. 

In [2]:
# TODO 2 - logo directory & name + scale size of logo
# TODO 2.1: get as input from user the logo path
# "C:\Users\mgaman\Downloads\logo-text.png"
logo_path = input("The path to the logo is... ")
# TODO 2.2: Associate an object that can be used in our program, with the image at this path
logo_initial_obj = Image.open(logo_path)
# TODO 2.3: Ask the user if it is upscale or down scale and by how much (width, height)
smaller = input("Will the logo be smaller than original? smaller/bigger/same : ")
w, h = logo_initial_obj.size
if smaller == "smaller":
    scale_factor_w = int(input("By how much to reduce the weight? "))
    scale_factor_h = int(input("By how much to reduce the height? "))
    w, h = w / scale_factor_w, h / scale_factor_h
elif smaller == "bigger":
    scale_factor_w = int(input("By how much to reduce the weight? "))
    scale_factor_h = int(input("By how much to reduce the height? "))
    w, h = w * scale_factor_w, h * scale_factor_h

# TODO 2.4: Build another object with the original logo resized to what's better for us
logo_obj = logo_initial_obj.resize((int(w), int(h)))

# TODO 2.5. - In which corner of the image will the logo be put?
corner = input("Where do you want the logo? [UP_LEFT]/DOWN_LEFT/UP_RIGHT/DOWN_RIGHT ")
if corner.upper not in ["UP_LEFT", "UP_RIGHT", "DOWN_LEFT", "DOWN_RIGHT"]:
    corner = "DOWN_RIGHT"

The path to the logo is... C:\Users\mgaman\Downloads\logo-text.png
Will the logo be smaller than original? smaller/bigger/same : SAME
Where do you want the logo? [UP_LEFT]/DOWN_LEFT/UP_RIGHT/DOWN_RIGHT DOWN_RIGHT


PASUL 3 - Adăugare funcții pentru determinarea poziției logo-ului în funcție de 
          dimensiunea sa și de cea a imaginii

In [3]:
# TODO 3 - Add a function which can determine a proper position of the left corner fo the logo, relative to
#          the image it will be put on
def get_corner_pos(w_logo, h_logo, w_img, h_img, corner):
    until_margin = int(min(w_logo, h_logo) / 4)
    pos_coords = {"UP_LEFT": (until_margin, until_margin),\
                 "UP_RIGHT": (w_img - w_logo - until_margin, until_margin), \
                 "DOWN_LEFT": (until_margin, h_img - h_logo - until_margin), \
                 "DOWN_RIGHT": (w_img - w_logo - until_margin, h_img - h_logo - until_margin)}
    return pos_coords[corner]
    

PASUL 4 - Unde salvez imaginile brand-uite?

In [4]:
# TODO 4 - Where do I save the new images?
# TODO 4.1 - set the path to the directory where to copy the modified images
# C:\\Users\\mgaman\\Pictures
to_dir_path = input("Introduce path to parent directory of destination folder: ")
# Change current directory to the value of this path
os.chdir(to_dir_path)

# TODO 4.2 - introduce directory name at this path. 
#            If the directory with the name introduced does not exist, create it.
to_dir_name = "pycodette_loggoed"
if not os.path.exists(to_dir_name):
    os.makedirs(to_dir_name)
# Change directory to the created dir
os.chdir(to_dir_name)

Introduce path to parent directory of destination folder: C:\\Users\\mgaman\\Pictures


Pasul 2: Salvăm imaginile găsite anterior, dar cu paste la logo în colțul indicat de user.

In [5]:
# TODO 5.1. - create a new list of image objects
img_processed_list = []
import copy

# TODO 5.2. - iterate through the list of images to be processed
# Hint: use for
for i in range(len(to_be_proc_list)):
    img = to_be_proc_list[i]
    img_new = copy.deepcopy(img)
    w_logo, h_logo = logo_obj.size
    w_img, h_img = img.size
    pos_new = get_corner_pos(w_logo, h_logo, w_img, h_img, corner)
    img_new.paste(logo_obj, pos_new, logo_obj)
    
    # Save the new image into the current working directory
    img_new.save(im_names[i])